In [15]:
import numpy as np
import scipy
import sympy
import data_import
import pandas as pd

# Data Import & Pre-processing

In [16]:
pd_data = data_import.read_files_to_pd_dataframe(
    [f"../outs/grid/itr_{i}.xml.out" for i in range(500)]
)

# Policy Iteration Algo

In [40]:

def get_policy(Q, x, b):
    """
    updates policy to the next policy,
    returns error
    """
    policy = []
    err2 = 0
    y = np.dot(Q, x) + b
    for i in range(len(x)):
        if y[i] < x[i]:
            policy.append(i)
        err2 += min(x[i], y[i]) ** 2    # x should be > 0, while y should be as close to 0 as possible
    return err2 ** 0.5, policy

# get the sub-system of linear equations given the policy
def get_sub_sys(policy, Q, b):
    policy_matrix = np.zeros((b.shape[0], b.shape[0]))
    for p in policy:
        policy_matrix[p,p] = 1
    I = np.eye(len(b))
    A = np.dot(
            np.dot(
                policy_matrix,
                Q
            ),
            policy_matrix
        ) + I - policy_matrix

    rhs = - policy_matrix @ b 
    # new_Q = np.zeros((len(policy), len(policy)))
    # new_b = np.zeros(len(policy))

    # for new_row, old_row in enumerate(policy):
    #     new_b[new_row] = b[old_row]
    #     for new_col, old_col in enumerate(policy):
    #         new_Q[new_row, new_col] = Q[old_row, old_col]
    
    return A, rhs
    


def update_value(policy, Q, b, use_cg = True):
    """
    returns the next x given the policy
    """
    I = np.eye(len(b))
    policy_matrix = np.zeros((b.shape[0], b.shape[0]))
    for p in policy:
        policy_matrix[p,p] = 1
    A = np.dot(
            np.dot(
                policy_matrix,
                Q
            ),
            policy_matrix
        ) + I - policy_matrix
    rhs = - policy_matrix @ b 
    lst_sqr_soln = np.linalg.lstsq(A , rhs)[0]
    cg_soln = scipy.sparse.linalg.cg(A , rhs , tol=1e-12)[0]
    if use_cg:
        return cg_soln
    else:
        return lst_sqr_soln


def flow(Q, N, v0, correct_policy, max_itrs = 100, m_tol = 1e-09, CoR = 1, use_cg = True):
    """
    does policy iteration
    returns a bool/vector pair: (converged, x)
    """
    b = np.dot(N.T, (1 + CoR) * v0)
    x = np.zeros(b.shape[0])
    policy = []
    error = 0.0

    found_correct_control = False

    for n_iter in range(max_itrs):

        # have we stumbled across the correct policy/control?
        if correct_policy == policy:
            found_correct_control = True

        error, policy = get_policy(Q, x, b)
        if error <= m_tol:
            return (True, x, found_correct_control)

        x = update_value(policy, Q, b, use_cg)
    
    return (False, x, found_correct_control)



# For sim 181...

policy iteration ends up hitting the "correct" policy (i.e. the policy that could feasibly result in the ipopt soln),
but the inner solver doesn't give a "good" soln, so we move off of the "correct" policy to a "worse?" one

In [18]:
def get_ipopt_control(x):
    # anything less than 1e-6 is counted as a 0 since thats the tolerance we set in the xml files
    return [i for i in range(len(x)) if x[i] > 1e-6]

pd_data['correct_control'] = [get_ipopt_control(x) for x in pd_data['ipopt_sol']]
# pd_data['correct_control']

In [47]:
i = 181
Q = pd_data['Q'][i]
N = pd_data['N'][i]
v0 = pd_data['v0'][i]

b = np.dot(N.T, (1 + 1) * v0)

correct_control = pd_data['correct_control'][i]

sub_Q, rhs = get_sub_sys(correct_control, Q, b)

np.set_printoptions(linewidth=1000, precision=6)
# scipy.linalg.null_space(sub_Q, 1e-5)  # null space is empty!
cg_soln = scipy.sparse.linalg.cg(sub_Q , rhs , tol=1e-12)[0]

ipopt_soln = pd_data['ipopt_sol'][i]


print(np.around(Q @ ipopt_soln + b, 5))


[ 0.       0.       0.16866  0.       0.11749  0.89797  0.12861  1.83571  1.41561 -0.      -0.       0.       1.2543   0.22435  0.04477  0.16199  0.16519  0.      -0.       1.05651 -0.       1.26802  0.       0.59757  1.73745  0.35604  0.     ]


it turns out that not even the ipopt solution is the "correct" solution!